# Duck Segmentation

This project builds an ML model that performs segmentation on the images that contain ducks.

The architecture used for segmentatic segmention is YOLO (YOLOv8).

---

## Image Labelling

This image labelling tool used is: CVAT.

For segmentation, the annotator used is primarily polygons, with coordinates available to download.

<img src='snapshots/image-annotation.png' width='70%' height='auto' />

With this, the **binary mask** is created for the respected image.

<img src='label_sample/masks/00eb7860432c771a.png' width='50%' height='auto' />

However, **label coordinates** are required for **YOLOv8**.

### From binary mask to label coordinates

In [19]:
import os
import cv2

mask_dir = os.path.join('.', 'label_samples', 'masks')
label_dir = os.path.join('.', 'label_samples', 'labels')

for file in os.listdir(mask_dir):
    mask_path = os.path.join(mask_dir, file)
    
    # Load binary mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
    
    mask_h, mask_w = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Converting contours to coordinates (polygons)
    coordinates = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            coordinate = []
            for point in cnt:
                x, y = point[0]
                coordinate.append(x / mask_w)
                coordinate.append(y / mask_h)
            coordinates.append(coordinate)
            
    # Printing coordinates
    with open('{}.txt'.format(os.path.join(label_dir, file)[:-4]), 'w') as f:
        for coordinate in coordinates:
            for p_, p in enumerate(coordinate):
                if p_ == len(coordinate) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

---

## Data preprocessing

The data structure for this project is:
* data/
    * images/
        * train/
        * val/
    * labels/
        * train/
        * val/